In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import constants
import itertools
import pandas as pd
from tabulate import tabulate
from matplotlib.lines import Line2D


%config InlineBackend.figure_format = 'svg'
plt.rcParams['lines.linewidth'] = 0.8

In [ ]:
epsilon_0 = constants.epsilon_0
c = constants.speed_of_light

fstart = 1
fstop = 10.6
fpoints = 300
f = np.logspace(fstart, fstop, fpoints)

In [ ]:
num_of_sections = 16
num_of_combinations = 2**num_of_sections

# Generate all possible combinations of an 8-bit input
input_combinations = list(itertools.product([0, 1], repeat=num_of_sections))

# Convert the combinations to a NumPy array
input_matrix = np.array(input_combinations)

In [ ]:
l = 6e-6/num_of_sections

In [ ]:
def invert_mat(A):
    inv_A = np.zeros((len(A[0, :, 0]), len(A[:, 0, 0]), len(A[0, 0, :])), dtype=complex)
    for i in range(len(A[0, 0])):
        inv_A[:, :, i] = np.linalg.inv(A[:, :, i])

    return inv_A

def multiply_mat(A, B):
    C = np.zeros((len(A[0, :, 0]), len(A[:, 0, 0]), len(A[0, 0, :])), dtype=complex)
    for i in range(len(A[0, 0, :])):
        C[:, :, i] = A[:, :, i] @ B[:, :, i]

    return C

In [ ]:
def odd_mode_params(f, Lo, Cm, Ca, Gm, Ga, R):
    w = 2*np.pi*f
    Co = 2*Cm - Ca
    Go = 2*Gm + Ga
    Lo = Lo
    Ro = R
    #gamma_o = 1e-5 + 1j*(2*np.pi*w*np.sqrt(Lo*Co))
    gamma_o = np.sqrt( (1j*w*Lo + Ro)*(1j*w*(Co) + Go) )
    Z0o = np.sqrt((Ro+1j*w*Lo)/(Go + 1j*w*Co))


    f0_o = 1/(2*np.pi*np.sqrt(Lo*Co)) # frequency at which the coupled lines are a quarter-wavelength long electrically when excited in the odd mode
    theta_o = (np.pi/2)*(f/f0_o)

    return Lo, Co, Go, Ro, gamma_o, Z0o, theta_o

def even_mode_params(f, Le, Cm, Ca, Gm, Ga, R):
    w = 2*np.pi*f
    Ce = Ca
    Ge = Ga
    Le = Le
    Re = R
    #gamma_e = 1e-4 + 1j*(2*np.pi*w*np.sqrt(Le*Ce))
    gamma_e = np.sqrt( (1j*w*Le + Re)*(1j*w*(Ce) + Ge) )
    Z0e = np.sqrt((Re+1j*w*Le)/(Ge + 1j*w*Ce))

    f0_e = 1/(2*np.pi*np.sqrt(Le*Ce)) #frequency at which the coupled lines are a quarter-wavelength long electrically when excited in the even mode,
    theta_e = (np.pi/2)*(f/f0_e)

    return Le, Ce, Ge, Re, gamma_e, Z0e, theta_e


In [ ]:
def find_Zin(ZL, Zc, gamma, l):
    return Zc * ( (ZL + 1j*Zc*np.tan(gamma*l))/(Zc + 1j*ZL*np.tan(gamma*l)) )

def Z0_even_odd_on_off(f, Lo, Le, Cm, Ca, Cst, Gm, Ga, R):
    # "Off" state calculation
    Lo_off, Co_off, Go_off, Ro_off, gamma_o_off, Z0o_off, theta_o_off = odd_mode_params(f, Lo, Cm, Ca, Gm, Ga, R)
    Le_off, Ce_off, Ge_off, Re_off, gamma_e_off, Z0e_off, theta_e_off = even_mode_params(f, Le, Cm, Ca, Gm, Ga, R)

    # "On" state calculations
    Lo_on, Co_on, Go_on, Ro_on, gamma_o_on, Z0o_on, theta_o_on = odd_mode_params(f, Lo, Cm + Cst, Ca, Gm, Ga, R)
    Le_on, Ce_on, Ge_on, Re_on, gamma_e_on, Z0e_on, theta_e_on = even_mode_params(f, Le, Cm, Ca, Gm, Ga, R)

    return Z0o_off, Z0o_on, Z0e_off, Z0e_on, gamma_o_off, gamma_o_on, gamma_e_off, gamma_e_on, theta_o_off, theta_o_on, theta_e_off, theta_e_on

def find_all_Zins(Z0o_off, Z0o_on, Z0e_off, Z0e_on, gamma_o_off, gamma_o_on, gamma_e_off, gamma_e_on, l, ZL):
    Zin_o_off = find_Zin(ZL, Z0o_off, gamma_o_off, l)
    Zin_o_on = find_Zin(ZL, Z0o_on, gamma_o_on, l)
    Zin_e_off = find_Zin(ZL, Z0e_off, gamma_e_off, l)
    Zin_e_on = find_Zin(ZL, Z0e_on, gamma_e_on, l)

    return Zin_o_off, Zin_o_on, Zin_e_off, Zin_e_on

def find_Zin_for_curr_section(num_of_sections, index, L, C_curr, gamma, l, f):
    Z01 = 50*np.ones(len(f))
    Z03 = Z01

    for i in range(0, index):
        Z01 = find_Zin(Z01, np.sqrt(L/C_curr[i]), gamma, l)
    
    for i in range(num_of_sections-1, index, -1):
        Z03 = find_Zin(Z03, np.sqrt(L/C_curr[i]), gamma, l)

    Z02 = Z01
    Z04 = Z03

    return Z01, Z02, Z03, Z04

In [ ]:
def create_A_matrix(gamma0_o, gamma0_e, Z0o, Z0e, l):
    Ao = np.zeros((2, 2, len(gamma0_o)), dtype=complex)
    Ae = np.zeros((2, 2, len(gamma0_e)), dtype=complex)

    Ao[0, 0, :] = np.cos(gamma0_o * l)
    Ao[0, 1, :] = 1j * Z0o * np.sin(gamma0_o * l)
    Ao[1, 0, :] = (1j / Z0o) * np.sin(gamma0_o * l)
    Ao[1, 1, :] = np.cos(gamma0_o * l)

    Ae[0, 0, :] = np.cos(gamma0_e * l)
    Ae[0, 1, :] = 1j * Z0e * np.sin(gamma0_e * l)
    Ae[1, 0, :] = (1j / Z0e) * np.sin(gamma0_e * l)
    Ae[1, 1, :] = np.cos(gamma0_e * l)

    return Ao, Ae


In [ ]:
def plot_S_params(f, S, title):
    gridSize = int(np.ceil(np.sqrt(4)))
    fig_S_params, ax = plt.subplots(gridSize, gridSize, figsize=(10, 10))
    for i in range(4):
        ax.flat[i].semilogx(f, 20*np.log10(np.abs(S[int(i/2), i%2, :])))
        ax.flat[i].grid(True)  # Add grid lines
    # Set a common title for all subplots
    fig_S_params.suptitle(title, fontsize=16)
    plt.show()

def plot_full_Z_params(f, Z, title):
    gridSize = int(np.ceil(np.sqrt(16)))
    fig_S_params, ax = plt.subplots(gridSize, gridSize, figsize=(15, 15))
    for i in range(16):
        row = int(i / 4)
        col = i % 4

        # Create a new y-axis on the right side of the current axes
        ax2 = ax.flat[i].twinx()

        # Plot the real part on the left y-axis and imaginary part on the right y-axis
        ax.flat[i].semilogx(f, np.real(Z[row, col, :]), label='Real')
        ax2.semilogx(f, np.imag(Z[row, col, :]), label='Imaginary', color='red')

        # Set labels and legends
        #ax.flat[i].set_ylabel('Real', color='blue')
        #ax2.set_ylabel('Imaginary', color='red')
        ax.flat[i].grid(True)
        
        # Add legends
        lines_1, labels_1 = ax.flat[i].get_legend_handles_labels()
        lines_2, labels_2 = ax2.get_legend_handles_labels()
        lines = lines_1 + lines_2
        labels = labels_1 + labels_2
        ax.flat[i].legend(lines, labels)

    # Set a common title for all subplots
    fig_S_params.suptitle(title, fontsize=16)
    plt.show()


def plot_full_S_params(f, S, title):
    gridSize = int(np.ceil(np.sqrt(16)))
    fig_S_params, ax = plt.subplots(gridSize, gridSize, figsize=(15, 15))
    for i in range(16):
        ax.flat[i].semilogx(f, 20*np.log10(np.abs(S[int(i/4), i%4, :])))
        ax.flat[i].grid(True)  # Add grid lines
    # Set a common title for all subplots
    fig_S_params.suptitle(title, fontsize=16)
    plt.show()

def plot_all_possible_S_params(f, S_storage, title, n):
    N = len(S_storage)
    gridSize = int(np.ceil(np.sqrt(4)))
    fig_S_params, ax = plt.subplots(gridSize, gridSize, figsize=(10, 8))

    # Create a list of legend entries
    legend_entries = []
    for j in range(0, N, int(N/n)):
        legend_entries.append(f"index={format(j, f'0{8}b')}")

    # Add a legend to each subplot
    for i in range(4):
        for j in range(0, N, int(N/n)):
            line, = ax.flat[i].semilogx(f, 20*np.log10(np.abs(S_storage[j][int(i/2), i%2, :])))
        ax.flat[i].grid(True)  # Add grid lines

    # Set a common title for all subplots
    fig_S_params.suptitle(title, fontsize=16)
    fig_S_params.legend(legend_entries)
    plt.show()


def plot_tan_csc(theta_o, theta_e):
    cot_theta_o = 1/np.tan(theta_o)
    cot_theta_e = 1/np.tan(theta_e)
    csc_theta_o = 1/np.sin(theta_o)
    csc_theta_e = 1/np.sin(theta_e)
    gridSize = int(np.ceil(np.sqrt(4)))
    fig_S_params, ax = plt.subplots(gridSize, gridSize, figsize=(10, 10))
    ax.flat[0].semilogx(theta_o, cot_theta_o)
    ax.flat[0].grid(True)  # Add grid lines
    ax.flat[1].semilogx(theta_e, cot_theta_e)
    ax.flat[1].grid(True)  # Add grid lines
    ax.flat[2].semilogx(theta_o, csc_theta_o)
    ax.flat[2].grid(True)  # Add grid lines
    ax.flat[3].semilogx(theta_e, cot_theta_e)
    ax.flat[3].grid(True)  # Add grid lines
    plt.show()


def plot_four_subplots_on_off(Ro_on, Ro_off, Lo_on, Lo_off, Go_on, Go_off, Co_on, Co_off):
    # Create a 2x2 grid of subplots
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    fig.suptitle("Four Subplots (On and Off)")

    # Plot on and off data in the subplots
    axes[0, 0].plot(Ro_on, label="On")
    axes[0, 0].plot(Ro_off, label="Off")
    axes[0, 0].set_title("Ro")
    axes[0, 0].legend()

    axes[0, 1].plot(Lo_on, label="On")
    axes[0, 1].plot(Lo_off, label="Off")
    axes[0, 1].set_title("Lo")
    axes[0, 1].legend()

    axes[1, 0].plot(Go_on, label="On")
    axes[1, 0].plot(Go_off, label="Off")
    axes[1, 0].set_title("Go")
    axes[1, 0].legend()

    axes[1, 1].plot(Co_on, label="On")
    axes[1, 1].plot(Co_off, label="Off")
    axes[1, 1].set_title("Co")
    axes[1, 1].legend()

    # Add labels and customize as needed
    for ax in axes.flat:
        ax.set(xlabel="X-label", ylabel="Y-label")

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)

    # Show the plots
    plt.show()

# Example usage:
# Replace Ro_on, Ro_off, Lo_on, Lo_off, Go_on, Go_off, Co_on, Co_off with your data arrays
# plot_four_subplots_on_off(Ro_on, Ro_off, Lo_on, Lo_off, Go_on, Go_off, Co_on, Co_off)


def plot_four_subplots_real_imag(Z01, Z02, Z03, Z04):
    # Create a 2x2 grid of subplots
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    fig.suptitle("Four Subplots (Real and Imaginary)")

    # Plot real and imaginary parts in the subplots
    axes[0, 0].plot(np.real(Z01), label="Real")
    axes[0, 0].plot(np.imag(Z01), label="Imaginary")
    axes[0, 0].set_title("Z01")
    axes[0, 0].legend()

    axes[0, 1].plot(np.real(Z02), label="Real")
    axes[0, 1].plot(np.imag(Z02), label="Imaginary")
    axes[0, 1].set_title("Z02")
    axes[0, 1].legend()

    axes[1, 0].plot(np.real(Z03), label="Real")
    axes[1, 0].plot(np.imag(Z03), label="Imaginary")
    axes[1, 0].set_title("Z03")
    axes[1, 0].legend()

    axes[1, 1].plot(np.real(Z04), label="Real")
    axes[1, 1].plot(np.imag(Z04), label="Imaginary")
    axes[1, 1].set_title("Z04")
    axes[1, 1].legend()

    # Add labels and customize as needed
    for ax in axes.flat:
        ax.set(xlabel="X-label", ylabel="Y-label")

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)

    # Show the plots
    plt.show()

# Example usage:
# Replace Z01, Z02, Z03, Z04 with your complex data arrays
# plot_four_subplots_real_imag(Z01, Z02, Z03, Z04)

